In [ ]:
import getpass
from datetime import datetime, timedelta
from tapipy.tapis import Tapis
from time import time_ns
import random
from random import randint
import pandas as pd
from io import StringIO
import matplotlib.pyplot as plt
import matplotlib.dates as md

# 1. Connect to Tapis
---
First well connect to Tapis and create a unique ID for this workshop. Make sure to use the username and password that you signed up for on the google sheets. This was sent out with the email for the workshop.

In [ ]:
username = getpass.getpass(prompt = "Username: ", stream = None)
password = getpass.getpass(prompt = "Password: ", stream = None)

base_url = "https://training.tapis.io"

client = Tapis(
    base_url = base_url, 
    username = username,
    password = password
) 

#generate access token
client.get_tokens()

In [ ]:
unique_id = f"{username}_{str(time_ns())[-10:]}"

In [ ]:
#Create a storage system in Tapis so we can upload and download data from our server
#NOTE you system id needs to be unique across Tapis
system_id = "training_tutorial_vm_" + unique_id
system_config = {
    "id": system_id,
    "description": "VM storage",
    "host": "149.165.153.42",
    "systemType": "LINUX",
    "defaultAuthnMethod": "PASSWORD",
    "effectiveUserId": username,
    "rootDir": f"/home/{username}/",
    "canExec": False
}

system = client.systems.createSystem(**system_config)

In [ ]:
# Add login credentials so Tapis can access the system - NORMALLY WE USED SSH KEYS but for this tutorial we will utilize the password auth
client.systems.createUserCredential(systemId = system_id,userName = username, password = password)

In [ ]:
#Create an Actor that we can pass to a channel to execute when a threshold triggers
actor = client.actors.create_actor(
    image = "mcleanj/workshop_actor",
    name = f"Plot Streams Data {unique_id}",
    description = "Actor that plots streams measurements.",
    default_environment = {"system_id": system_id, "destination_path": "/"}
)

In [ ]:
client.actors.get_actor(actor_id = actor.id)

In [ ]:
#Create a Channel to check for our Temperature exceeding 100- then execute an Actor to generate a plot and upload to our system
channel_id = f"demo.tapis.channel-{unique_id}"
channel = client.streams.create_channels(channel_id=channel_id, 
                            channel_name=channel_id, 
                            template_id="default_threshold",
                            triggers_with_actions=[
                                {"inst_ids":[inst_id],
                                "condition":{"key":inst_id+".temperature",
                                              "operator":">", 
                                              "val":100},
                                 "action":{"method":"ACTOR","actor_id" : my_actor.id,
                                           "message":"Instrument: exceeded Temperature threshold"}}])
channel

In [ ]:
client.streams.list_channels()

In [ ]:
#create measurement to trigger channel
variables = [{"temperature": 230,
    "rainfall": 0,
    "datetime": datetime.now().isoformat()
}]
#write observations to measurements endpoint for our instrument
result = client.streams.create_measurement(inst_id=inst_id, vars=variables)
result

In [ ]:
#List the Alerts issued on our Channel
client.streams.list_alerts(channel_id=channel.channel_id)

In [ ]:
#Fetch the latest alert and assign to a variable
alert = client.streams.list_alerts(channel_id=channel.channel_id).alerts[0]

In [ ]:
#Fetch our Actor execution log
client.actors.get_execution_logs(actor_id=alert.actor_id, 
                                 execution_id=alert.execution_id)